In [ ]:
import numpy as np
import os

file_list = os.listdir('/content/SR250Breath')
windows = []
labels = []

for file_name in file_list:
    file_path = os.path.join('/content/SR250Breath', file_name)
    try:
        data = np.load(file_path)

        if file_path.endswith('.window.npy'):
            windows.append(data)
        elif file_path.endswith('.label.npy'):
            labels.append(data)
        else:
            print(f"Unknown file type: {file_path}")
    except Exception as e:
        print(f"Could not load {file_name}: {e}")

# You can access the arrays using the file names as keys, for example:
# print(data_arrays['Millenials_E_breath_sitting_desk_20250909-115853.window.npy'])

In [ ]:
windows = np.concat(windows)
labels = np.concat(labels)

print(windows.shape)
print(labels.shape)

In [ ]:
# in maniera tale che l'ultimo asse corrisponde alle antenne
windows = windows.transpose(0, 1, 3, 2)
print(windows.shape)

In [ ]:
abs_windows = np.abs(windows)
print(abs_windows.shape)

In [ ]:
phase_windows = np.angle(windows)

In [ ]:
all = np.concatenate((abs_windows, phase_windows), axis=-1)
# all = phase_windows
print(all.shape)

In [ ]:
import tensorflow as tf

# Assuming 'labels' is your numpy array of labels
# You might need to adjust the 'depth' parameter based on the number of unique classes in your labels
one_hot_labels = tf.one_hot(labels, depth=2) # Assuming 2 classes for demonstration

print(one_hot_labels.shape)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Convert TensorFlow tensor to NumPy array
one_hot_labels_np = one_hot_labels.numpy()

# Split into training and temporary sets (for test and validation)
X_train, X_temp, y_train, y_temp = train_test_split(phase_windows, one_hot_labels_np, test_size=0.3, random_state=42)

# Split the temporary set into test and validation sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training set shapes:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("\nValidation set shapes:")
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)
print("\nTest set shapes:")
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

# Task
Produce a CNN model for the prediction of the label and train it using the training, test, and validation data split from the `abs_windows` and `labels` variables.

## Define the cnn model architecture

### Subtask:
Define the layers of the CNN model, including convolutional layers, pooling layers, and dense layers, suitable for your data shape.


**Reasoning**:
Define the CNN model architecture using convolutional, pooling, and dense layers.



**Reasoning**:
The error indicates that the input shape is too small for the convolutional and pooling layers. Adjust the kernel size and pooling size to be smaller to avoid negative dimensions.



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with adjusted kernel and pooling sizes
model.add(Conv2D(16, (1, 1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (1, 1), activation='relu'))
model.add(MaxPooling2D((2, 4)))

# Flatten the output
model.add(Flatten())

# Add dense layers
# model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_little = model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with adjusted kernel and pooling sizes
model.add(Conv2D(32, (1, 1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (1, 1), activation='relu'))
model.add(MaxPooling2D((2, 4)))

# Flatten the output
model.add(Flatten())

# Add dense layers
# model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_small = model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with adjusted kernel and pooling sizes
model.add(Conv2D(32, (2, 2), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((1, 2))) # Pool only along the second dimension
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 2)))
model.add(Conv2D(128, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 2)))


# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_medium = model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with further adjusted kernel and pooling sizes
model.add(Conv2D(32, (1, 1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((1, 1))) # Use (1,1) pooling to avoid reducing dimension too quickly
model.add(Conv2D(64, (1, 1), activation='relu'))
model.add(MaxPooling2D((1, 1)))
model.add(Conv2D(128, (1, 1), activation='relu'))
model.add(MaxPooling2D((1, 1)))


# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_large = model

**Reasoning**:
The model summary shows that the output shape after the last MaxPooling2D layer has a dimension of 0, which is incorrect. The input shape is likely still too large for the kernel and pooling sizes. I need to further adjust the kernel and pooling sizes to ensure the output dimensions remain positive.



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with adjusted kernel and pooling sizes
model.add(Conv2D(32, (1, 1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (1, 1), activation='relu'))
model.add(MaxPooling2D((2, 4)))


# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_small_test1 = model

**Reasoning**:
The model summary now shows positive dimensions after the pooling layers, but using (1,1) kernel and pooling sizes essentially bypasses the spatial feature extraction capabilities of CNNs. I need to find a balance between kernel/pooling sizes and the input shape to effectively use convolutional layers without causing negative dimensions. I will try increasing the kernel size slightly while keeping the pooling size small.



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with adjusted kernel and pooling sizes
model.add(Conv2D(32, (2, 2), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((1, 2))) # Pool only along the second dimension
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 2)))
model.add(Conv2D(128, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 2)))


# Flatten the output
model.add(Flatten())

# Add dense layers
# model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_small_test2 = model

## Compile the model

### Subtask:
Compile the CNN model by specifying the optimizer, loss function, and metrics.


**Reasoning**:
Compile the CNN model with the Adam optimizer, binary crossentropy loss, and accuracy metric.



In [ ]:
from tensorflow.keras.optimizers import Adam

model = model_small_test2

model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Train the model

### Subtask:
Train the compiled model using the training data (`X_train`, `y_train`) and validate it using the validation data (`X_val`, `y_val`).


**Reasoning**:
Train the compiled model using the training and validation data.



In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

**Reasoning**:
The first step is to load the data from the CSV file into a pandas DataFrame and display the first few rows to understand its structure.



## Evaluate the model

### Subtask:
Evaluate the trained model on the test data (`X_test`, `y_test`) to assess its performance.

**Reasoning**:
Evaluate the trained model on the test data to assess its performance.

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Get predictions from the model
y_pred_prob = model.predict(X_test)

# Convert probabilities to class predictions (assuming binary classification and sigmoid activation)
# Since y_test is one-hot encoded, y_pred should also be one-hot encoded for comparison with some metrics
# However, precision_score, recall_score, and f1_score with average='binary' expect binary labels.
# Let's convert both to binary labels for these metrics.
y_pred = (y_pred_prob[:, 1] > 0.5).astype(int) # Assuming the second column is the positive class probability

# Convert one-hot encoded true labels back to single class labels
y_true = np.argmax(y_test, axis=1)

# Calculate metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

In [ ]:
import tensorflow as tf

# Quantize the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save the quantized model (optional)
# with open('quantized_model.tflite', 'wb') as f:
#     f.write(quantized_tflite_model)

# Load the quantized model
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare test data for inference
# TFLite models expect float32 inputs
X_test_float32 = X_test.astype(np.float32)

# Run inference on the test set
predictions = []
for i in range(X_test_float32.shape[0]):
    interpreter.set_tensor(input_details[0]['index'], np.expand_dims(X_test_float32[i], axis=0))
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions.append(output_data[0])

predictions = np.array(predictions)

# Evaluate the quantized model (assuming y_test is already one-hot encoded)
# For binary classification with sigmoid output, you can use binary_accuracy
from tensorflow.keras.metrics import BinaryAccuracy

binary_accuracy = BinaryAccuracy()
binary_accuracy.update_state(y_test, predictions)

print(f"Quantized Model Test Accuracy: {binary_accuracy.result().numpy()}")

# If you want to calculate loss, you'd need to implement it manually or convert predictions to match y_test format
# For example, if y_test is one-hot, and predictions are probabilities for each class:
# from tensorflow.keras.losses import BinaryCrossentropy
# bce = BinaryCrossentropy()
# loss = bce(y_test, predictions).numpy()
# print(f"Quantized Model Test Loss: {loss}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# predictions from the quantized model were already calculated in cell SUFu4NnP0VZO

# Convert probabilities to class predictions (assuming binary classification and sigmoid activation)
y_pred_quantized = (predictions[:, 1] > 0.5).astype(int) # Assuming the second column is the positive class probability

# Convert one-hot encoded true labels back to single class labels
y_true = np.argmax(y_test, axis=1)

# Calculate metrics for the quantized model
precision_quantized = precision_score(y_true, y_pred_quantized)
recall_quantized = recall_score(y_true, y_pred_quantized)
f1_quantized = f1_score(y_true, y_pred_quantized)

print(f"Quantized Model Precision: {precision_quantized}")
print(f"Quantized Model Recall: {recall_quantized}")
print(f"Quantized Model F1 Score: {f1_quantized}")

In [ ]:
import os

# Save the quantized model temporarily to check its size
quantized_model_path = 'quantized_model.tflite'
with open(quantized_model_path, 'wb') as f:
    f.write(quantized_tflite_model)

# Get the file size in bytes and convert to kilobytes
model_size_bytes = os.path.getsize(quantized_model_path)
model_size_kb = model_size_bytes / 1024

print(f"Dimensioni del modello quantizzato: {model_size_bytes} bytes ({model_size_kb:.2f} KB)")

# Clean up the temporary file
os.remove(quantized_model_path)

In [ ]:
os.mkdir('/content/models')

model.save(os.path.join('/content/models', 'model_small_test2' + '2.keras'))

In [ ]:
# Float model export:
# The second argument, string format, must be the model name + 2.keras
model_small_test2 = tf.keras.models.load_model(os.path.join('/content/models', 'model_small_test22.keras'))

converter = tf.lite.TFLiteConverter.from_keras_model(model_small_test2)
tflite_model = converter.convert()
print("Float model size:", open(os.path.join('/content/models', 'model_small_test2' + '.tflite'), "wb").write(tflite_model))

In [ ]:
print(X_train[0].shape)

In [ ]:
# Quantized model export:

# Definition of Representative Dataset generator:
def representative_data_gen():
  for i in range(X_train.shape[0]):
    yield [X_train[i].reshape((-1,) + X_train[i].shape)]

#def representative_dataset():
#  for i in range(100):
#    yield [ np.array([(np.random.rand(1960)).astype(np.float32)]) ]

converter = tf.lite.TFLiteConverter.from_keras_model(model_small_test2)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.compat.v1.lite.constants.INT8 # or tf.uint8
converter.inference_output_type = tf.compat.v1.lite.constants.INT8  # or tf.uint8

tflite_model_quant = converter.convert()
print("Quantized model size: ", open(os.path.join('/content/models', 'model_small_test2' + '-int8.tflite'), "wb").write(tflite_model_quant))

In [ ]:
!apt-get update && apt-get -qq install xxd

MODEL_TFLITE = '/content/models/'+ 'model_small_test2' +'-int8.tflite'
MODEL_TFLITE_MICRO = 'TinyConvModel-int8.cc'

In [ ]:
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}